In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install accelerate>=0.21.0
!pip install torch

In [ ]:
from datasets import load_dataset
from datasets import concatenate_datasets
from transformers import AutoTokenizer

In [ ]:
model_name = """your model name"""

tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_fast=True,
        trust_remote_code=True,
    )

Merge dolly-ja

In [ ]:
from datasets import load_dataset
dataset_name ="kunishou/databricks-dolly-15k-ja"
dataset = load_dataset(dataset_name)['train']
df = pd.DataFrame(dataset)
print(df)

In [ ]:
df = dataset
system_message = f"以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
response_template = "\n\n### 応答:\n"
input_template = "\n\n### 入力:\n"

def merge(examples):
    if pd.isna(examples['input']) or examples['input'] == '':
        return system_message + examples['instruction'] + response_template + str(examples['output'])
    else:
        return system_message + examples['instruction'] + input_template + examples['input'] + response_template + str(examples['output'])

df['text'] = df.apply(merge, axis=1)

In [ ]:
output_path=f"/content/drive/MyDrive/GENIAC/" + "dolly-15k-jp-merged.jsonl"
import json
with open(output_path, 'w') as f:
    for id, row in df.iterrows():
        f.write(json.dumps({"text": row['text']}, ensure_ascii=False)+"\n")
dataset = load_dataset("json", data_files=output_path)

In [ ]:
import json
name=output_path
dataset = load_dataset("json", data_files=name)
no_filter = dataset.num_rows
print(f"Number of rows in {dataset.num_rows}")
dataset_filltered = dataset.filter(lambda x: len(tokenizer.encode(x['text'])) < 2045)
print(f"Number of filltered rows in {dataset_filltered .num_rows}")
if no_filter != dataset_filltered.num_rows:
    output_path=f"/content/drive/MyDrive/GENIAC/" + name.split("/")[-1].split('.')[0]  + "-filtered.jsonl"
    dataset_filltered = pd.DataFrame(dataset_filltered['train'])
    import json
    with open(output_path, 'w') as f:
        for id, row in dataset_filltered.iterrows():
            f.write(json.dumps({"text": row['text']}, ensure_ascii=False)+"\n")
    dataset = load_dataset("json", data_files=output_path)
    print(dataset["train"][0:10])
else:
    print(f"No filltered rows in {dataset.num_rows}")

##oasst2-32k-ja

In [ ]:
from datasets import load_dataset
dataset_name ="llm-jp/oasst2-33k-ja"
Oasst_21k_dataset = load_dataset(dataset_name)['train']
df = pd.DataFrame(Oasst_21k_dataset)
print(df)

In [ ]:
system_message = f"以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
human_template = "\n\n### 指示:\n"
response_template = "\n\n### 応答:\n"

def merge(examples):
    messages = system_message
    for i in range(len(examples['conversations'])):
        if examples['conversations'][i]['role'] == "user":
            if i!=0:
                messages += human_template
            messages += examples['conversations'][i]['content']
        elif i < len(examples['conversations']) - 1:
            messages += response_template + examples['conversations'][i]['content'] +"</s>"
        else:
            messages += response_template + examples['conversations'][i]['content']
    return messages

df['text'] = df.apply(merge, axis=1)
df['conversations'] = ''
df['text'].iloc[1]

In [ ]:
output_path=f"/content/drive/MyDrive/GENIAC/" + "oasst2-33k-en-merged-sp.jsonl"
import json
with open(output_path, 'w') as f:
    for id, row in df.iterrows():
        f.write(json.dumps({"text": row['text']}, ensure_ascii=False)+"\n")
dataset = load_dataset("json", data_files=output_path)
print(dataset["train"][0])

In [ ]:
import json
name=output_path
dataset = load_dataset("json", data_files=name)
no_filter = dataset.num_rows
print(f"Number of rows in {dataset.num_rows}")
dataset_filltered = dataset.filter(lambda x: len(tokenizer.encode(x['text'])) < 2045)
print(f"Number of filltered rows in {dataset_filltered .num_rows}")
if no_filter != dataset_filltered.num_rows:
    output_path=f"/content/drive/MyDrive/GENIAC/" + name.split("/")[-1].split('.')[0]  + "-filtered.jsonl"
    dataset_filltered = pd.DataFrame(dataset_filltered['train'])
    import json
    with open(output_path, 'w') as f:
        for id, row in dataset_filltered.iterrows():
            f.write(json.dumps({"text": row['text']}, ensure_ascii=False)+"\n")
    dataset = load_dataset("json", data_files=output_path)
    print(dataset["train"][0:10])
else:
    print(f"No filltered rows in {dataset.num_rows}")